# Notebook and Module Parsing

> Parse notebook metadata, content, and extract function/class signatures with docments

In [ ]:
#| default_exp parsers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple, Union
import ast
import re
from dataclasses import dataclass, field
from nbdev.config import get_config
from cjm_nbdev_overview.core import *
from cjm_nbdev_overview.tree import extract_notebook_info
import inspect
from fastcore.imports import *

## Data Models for Parsing

In [ ]:
#| export
@dataclass
class FunctionInfo:
    "Information about a function"
    name: str                                      # Function name
    signature: str                                 # Full signature with docments
    docstring: Optional[str] = None               # Function docstring
    decorators: List[str] = field(default_factory=list)  # List of decorators
    is_exported: bool = False                     # Has #| export
    is_async: bool = False                        # Is an async function
    source_line: Optional[int] = None             # Line number in source

In [ ]:
#| export
@dataclass
class VariableInfo:
    "Information about a module-level variable"
    name: str                                      # Variable name
    value: Optional[str] = None                   # String representation of value
    type_hint: Optional[str] = None               # Type annotation if present
    comment: Optional[str] = None                 # Inline comment
    is_exported: bool = False                     # Has #| export

In [ ]:
#| export
@dataclass
class ClassInfo:
    "Information about a class"
    name: str                                      # Class name
    signature: str                                 # Class signature with __init__
    docstring: Optional[str] = None               # Class docstring
    methods: List[FunctionInfo] = field(default_factory=list)  # Class methods
    decorators: List[str] = field(default_factory=list)        # Class decorators
    attributes: List[VariableInfo] = field(default_factory=list)  # Class attributes (for dataclasses)
    is_exported: bool = False                     # Has #| export
    source_line: Optional[int] = None             # Line number in source

In [ ]:
#| export
@dataclass
class ModuleInfo:
    "Information about a module (notebook or Python file)"
    path: Path                                     # Path to module
    name: str                                      # Module name
    title: Optional[str] = None                   # H1 title from notebook
    description: Optional[str] = None             # Module description
    functions: List[FunctionInfo] = field(default_factory=list)  # Functions in module
    classes: List[ClassInfo] = field(default_factory=list)       # Classes in module
    variables: List[VariableInfo] = field(default_factory=list)  # Variables in module
    imports: List[str] = field(default_factory=list)             # Import statements

## AST Parsing Utilities

In [ ]:
#| export
def extract_docments_signature(node: Union[ast.FunctionDef, ast.AsyncFunctionDef],  # AST function node
                              source_lines: List[str]                               # Source code lines
                              ) -> str:                                             # Function signature
    "Extract function signature with docments-style comments"
    # Get the function definition lines
    start_line = node.lineno - 1
    end_line = node.lineno - 1
    
    # Find the end of the function signature (look for ':')
    paren_count = 0
    in_string = False
    string_char = None
    
    for i in range(start_line, min(start_line + 20, len(source_lines))):
        line = source_lines[i]
        for j, char in enumerate(line):
            if in_string:
                if char == string_char and (j == 0 or line[j-1] != '\\'):
                    in_string = False
            else:
                if char in '"\'':
                    in_string = True
                    string_char = char
                elif char == '(':
                    paren_count += 1
                elif char == ')':
                    paren_count -= 1
                elif char == ':' and paren_count == 0:
                    end_line = i
                    break
        if end_line > start_line:
            break
    
    # Extract the signature lines
    if end_line >= start_line:
        sig_lines = source_lines[start_line:end_line + 1]
        signature = '\n'.join(sig_lines).strip()
        
        # Clean up the signature
        if signature.endswith(':'):
            signature = signature[:-1].strip()
        
        # Ensure async keyword is preserved if present
        if isinstance(node, ast.AsyncFunctionDef) and not signature.startswith('async '):
            signature = 'async ' + signature
        
        return signature
    
    # Fallback to basic signature
    async_prefix = "async " if isinstance(node, ast.AsyncFunctionDef) else ""
    return f"{async_prefix}def {node.name}(...)"

In [ ]:
#| export
def _parse_decorators(node: Union[ast.ClassDef, ast.FunctionDef, ast.AsyncFunctionDef]  # AST node with decorators
                     ) -> List[str]:                                                    # List of decorator names
    "Parse decorators from an AST node"
    decorators = []
    for decorator in node.decorator_list:
        if isinstance(decorator, ast.Name):
            decorators.append(decorator.id)
        elif isinstance(decorator, ast.Attribute):
            decorators.append(f"{decorator.attr}")
    return decorators

In [ ]:
#| export
def parse_function(node: Union[ast.FunctionDef, ast.AsyncFunctionDef],  # AST function node
                  source_lines: List[str],                             # Source code lines
                  is_exported: bool = False                            # Has #| export
                  ) -> FunctionInfo:                                   # Function information
    "Parse a function definition from AST"
    # Extract signature with docments
    signature = extract_docments_signature(node, source_lines)
    
    # Get docstring
    docstring = ast.get_docstring(node)
    
    # Get decorators
    decorators = _parse_decorators(node)
    
    # Check if it's an async function
    is_async = isinstance(node, ast.AsyncFunctionDef)
    
    return FunctionInfo(
        name=node.name,
        signature=signature,
        docstring=docstring,
        decorators=decorators,
        is_exported=is_exported,
        is_async=is_async,
        source_line=node.lineno
    )

In [ ]:
#| export
def _parse_class_methods(node: ast.ClassDef,           # AST class node
                        source_lines: List[str],        # Source code lines
                        is_exported: bool = False       # Has #| export
                        ) -> List[FunctionInfo]:        # List of method information
    "Parse methods from a class definition"
    methods = []
    for item in node.body:
        if isinstance(item, (ast.FunctionDef, ast.AsyncFunctionDef)):
            method_info = parse_function(item, source_lines, is_exported)
            methods.append(method_info)
    return methods

In [ ]:
#| export
def _parse_dataclass_attributes(node: ast.ClassDef,    # AST class node
                               source_lines: List[str], # Source code lines
                               is_exported: bool = False # Has #| export
                               ) -> List[VariableInfo]: # List of attribute information
    "Parse dataclass attributes from a class definition"
    attributes = []
    for item in node.body:
        if isinstance(item, ast.AnnAssign) and isinstance(item.target, ast.Name):
            # This is a type-annotated attribute
            attr_name = item.target.id
            type_hint = ast.unparse(item.annotation) if hasattr(ast, 'unparse') else str(item.annotation)
            
            # Get the default value if present
            default_value = None
            if item.value:
                try:
                    if isinstance(item.value, ast.Constant):
                        default_value = repr(item.value.value)
                    elif isinstance(item.value, ast.Call):
                        # Handle field() calls
                        if isinstance(item.value.func, ast.Name) and item.value.func.id == 'field':
                            default_value = "field(...)"
                        else:
                            default_value = ast.unparse(item.value) if hasattr(ast, 'unparse') else "..."
                    else:
                        default_value = ast.unparse(item.value) if hasattr(ast, 'unparse') else "..."
                except:
                    default_value = "..."
            
            # Get inline comment
            comment = None
            if item.lineno <= len(source_lines):
                line = source_lines[item.lineno - 1]
                comment_match = re.search(r'#\s*(.+)$', line)
                if comment_match:
                    comment = comment_match.group(1).strip()
            
            attr_info = VariableInfo(
                name=attr_name,
                type_hint=type_hint,
                value=default_value,
                comment=comment,
                is_exported=is_exported
            )
            attributes.append(attr_info)
    return attributes

In [ ]:
#| export
def _generate_class_signature(node: ast.ClassDef,      # AST class node
                             methods: List[FunctionInfo] # List of class methods
                             ) -> str:                  # Class signature
    "Generate a class signature including __init__ if present"
    # Start with basic class signature
    class_sig = f"class {node.name}"
    
    # Add base classes if present
    if node.bases:
        base_names = []
        for base in node.bases:
            if isinstance(base, ast.Name):
                base_names.append(base.id)
            elif isinstance(base, ast.Attribute):
                base_names.append(f"{base.attr}")
        if base_names:
            class_sig += f"({', '.join(base_names)})"
    
    class_sig += ":"
    
    # If there's an __init__ method, include its signature
    init_method = next((m for m in methods if m.name == "__init__"), None)
    if init_method:
        class_sig = f"class {node.name}:\n    {init_method.signature}"
    
    return class_sig

In [ ]:
#| export
def parse_class(node: ast.ClassDef,                    # AST class node
               source_lines: List[str],                # Source code lines
               is_exported: bool = False               # Has #| export
               ) -> ClassInfo:                         # Class information
    "Parse a class definition from AST"
    # Get class docstring
    docstring = ast.get_docstring(node)
    
    # Get decorators
    decorators = _parse_decorators(node)
    
    # Parse methods
    methods = _parse_class_methods(node, source_lines, is_exported)
    
    # Extract dataclass attributes if this is a dataclass
    attributes = []
    is_dataclass = 'dataclass' in decorators
    
    if is_dataclass:
        attributes = _parse_dataclass_attributes(node, source_lines, is_exported)
    
    # Generate class signature
    class_sig = _generate_class_signature(node, methods)
    
    return ClassInfo(
        name=node.name,
        signature=class_sig,
        docstring=docstring,
        methods=methods,
        decorators=decorators,
        attributes=attributes,
        is_exported=is_exported,
        source_line=node.lineno
    )

In [ ]:
#| export
def parse_variable(node: Union[ast.Assign, ast.AnnAssign],    # AST assignment node
                  source_lines: List[str],                     # Source code lines
                  is_exported: bool = False                    # Has #| export
                  ) -> List[VariableInfo]:                     # Variable information
    "Parse variable assignments from AST"
    variables = []
    
    if isinstance(node, ast.Assign):
        # Regular assignment
        for target in node.targets:
            if isinstance(target, ast.Name):
                # Try to get the value representation
                value_str = None
                if node.value:
                    try:
                        if isinstance(node.value, (ast.Constant, ast.Num, ast.Str)):
                            value_str = repr(node.value.value if hasattr(node.value, 'value') else node.value.n if hasattr(node.value, 'n') else node.value.s)
                        elif isinstance(node.value, ast.List):
                            value_str = f"[{len(node.value.elts)} items]"
                        elif isinstance(node.value, ast.Dict):
                            value_str = "{" + f"{len(node.value.keys)} items" + "}"
                    except:
                        pass
                
                # Check for inline comment
                comment = None
                if node.lineno <= len(source_lines):
                    line = source_lines[node.lineno - 1]
                    comment_match = re.search(r'#\s*(.+)$', line)
                    if comment_match:
                        comment = comment_match.group(1).strip()
                
                variables.append(VariableInfo(
                    name=target.id,
                    value=value_str,
                    comment=comment,
                    is_exported=is_exported
                ))
    
    elif isinstance(node, ast.AnnAssign):
        # Annotated assignment
        if isinstance(node.target, ast.Name):
            type_hint = ast.unparse(node.annotation) if hasattr(ast, 'unparse') else None
            
            value_str = None
            if node.value:
                try:
                    if isinstance(node.value, (ast.Constant, ast.Num, ast.Str)):
                        value_str = repr(node.value.value if hasattr(node.value, 'value') else node.value.n if hasattr(node.value, 'n') else node.value.s)
                except:
                    pass
            
            variables.append(VariableInfo(
                name=node.target.id,
                value=value_str,
                type_hint=type_hint,
                is_exported=is_exported
            ))
    
    return variables

## Notebook Cell Parsing

In [ ]:
#| export
def parse_code_cell(cell: Dict[str, Any]                       # Notebook code cell
                   ) -> Tuple[List[FunctionInfo], List[ClassInfo], List[VariableInfo], List[str]]:  # (functions, classes, variables, imports)
    """Parse a notebook code cell for functions, classes, variables, and imports"""
    source = get_cell_source(cell)
    source_lines = source.split('\n')
    
    # Check if cell is exported - support all nbdev export directive formats
    # Supports: #| export, #|export, #| exporti, #|exporti, #| exports, #|exports
    is_exported = any(re.match(r'^\s*#\|\s*export[is]?\s*$', line) for line in source_lines)
    
    functions = []
    classes = []
    variables = []
    imports = []
    
    try:
        # Parse the AST
        tree = ast.parse(source)
        
        # Only process top-level nodes
        for node in tree.body:
            if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
                func_info = parse_function(node, source_lines, is_exported)
                functions.append(func_info)
            
            elif isinstance(node, ast.ClassDef):
                class_info = parse_class(node, source_lines, is_exported)
                classes.append(class_info)
            
            elif isinstance(node, (ast.Assign, ast.AnnAssign)):
                # Only include uppercase variables (constants) or type-annotated variables
                var_infos = parse_variable(node, source_lines, is_exported)
                for var in var_infos:
                    # Filter out common local variables and dataclass fields
                    if (var.name.isupper() or  # Constants like DEFAULT_CONFIG
                        var.type_hint or  # Type annotated variables
                        (var.comment and is_exported)):  # Variables with meaningful comments
                        # Skip dataclass field definitions
                        if not (var.value and 'field(' in str(var.value)):
                            variables.append(var)
            
            elif isinstance(node, (ast.Import, ast.ImportFrom)):
                # Extract import statements
                if isinstance(node, ast.Import):
                    for alias in node.names:
                        imports.append(f"import {alias.name}")
                else:
                    module = node.module or ''
                    for alias in node.names:
                        if module:
                            imports.append(f"from {module} import {alias.name}")
                        else:
                            imports.append(f"import {alias.name}")
    
    except SyntaxError:
        # If we can't parse the cell, skip it
        pass
    
    return functions, classes, variables, imports

## Module Parsing

In [ ]:
#| export
def parse_notebook(path: Path                           # Path to notebook
                  ) -> ModuleInfo:                      # Module information
    "Parse a notebook file for module information"
    # Extract basic notebook info
    nb_info = extract_notebook_info(path)
    
    # Create module info
    module_info = ModuleInfo(
        path=path,
        name=nb_info.export_module or path.stem,
        title=nb_info.title,
        description=nb_info.description
    )
    
    # Read notebook content
    nb_content = read_notebook(path)
    cells = nb_content.get('cells', [])
    
    # Parse each code cell
    for cell in cells:
        if cell.get('cell_type') == 'code':
            funcs, classes, vars, imports = parse_code_cell(cell)
            module_info.functions.extend(funcs)
            module_info.classes.extend(classes)
            module_info.variables.extend(vars)
            module_info.imports.extend(imports)
    
    # Remove duplicate imports
    module_info.imports = list(set(module_info.imports))
    
    return module_info

In [ ]:
#| export
def parse_python_file(path: Path                        # Path to Python file
                     ) -> ModuleInfo:                   # Module information
    "Parse a Python file for module information"
    module_info = ModuleInfo(
        path=path,
        name=path.stem
    )
    
    # Read file content
    with open(path, 'r', encoding='utf-8') as f:
        source = f.read()
        source_lines = source.split('\n')
    
    # Look for module docstring and description
    try:
        tree = ast.parse(source)
        module_docstring = ast.get_docstring(tree)
        if module_docstring:
            lines = module_docstring.strip().split('\n')
            if lines:
                module_info.title = lines[0]
                if len(lines) > 1:
                    module_info.description = lines[1].strip('> ')
        
        # Parse all top-level definitions
        for node in tree.body:
            if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
                func_info = parse_function(node, source_lines, True)
                module_info.functions.append(func_info)
            
            elif isinstance(node, ast.ClassDef):
                class_info = parse_class(node, source_lines, True)
                module_info.classes.append(class_info)
            
            elif isinstance(node, (ast.Assign, ast.AnnAssign)):
                var_infos = parse_variable(node, source_lines, True)
                module_info.variables.extend(var_infos)
            
            elif isinstance(node, (ast.Import, ast.ImportFrom)):
                if isinstance(node, ast.Import):
                    for alias in node.names:
                        module_info.imports.append(f"import {alias.name}")
                else:
                    module = node.module or ''
                    for alias in node.names:
                        if module:
                            module_info.imports.append(f"from {module} import {alias.name}")
    
    except SyntaxError:
        pass
    
    return module_info

## Testing

Let's test the parser on our own notebooks:

In [ ]:
# Test parsing the core module
core_info = parse_notebook(Path("core.ipynb"))
print(f"Module: {core_info.name}")
print(f"Title: {core_info.title}")
print(f"Description: {core_info.description}")
print(f"\nFunctions ({len(core_info.functions)}):")
for func in core_info.functions[:3]:  # Show first 3
    print(f"  - {func.name}")
print(f"\nClasses ({len(core_info.classes)}):")
for cls in core_info.classes[:3]:  # Show first 3
    print(f"  - {cls.name}")
    
print(f"\nTesting refactored parse_class function...")
print("Class signatures:")
for cls in core_info.classes:
    print(f"\n{cls.name}:")
    print(f"  Decorators: {cls.decorators}")
    print(f"  Methods: {[m.name for m in cls.methods]}")
    print(f"  Attributes: {[a.name for a in cls.attributes]}")
    print(f"  Signature: {cls.signature[:100]}...")  # First 100 chars

Module: core
Title: Core Utilities
Description: Core utilities and data models for nbdev project overview generation

Functions (4):
  - get_notebook_files
  - get_subdirectories
  - read_notebook

Classes (2):
  - NotebookInfo
  - DirectoryInfo

Testing refactored parse_class function...
Class signatures:

NotebookInfo:
  Decorators: ['dataclass']
  Methods: ['relative_path']
  Attributes: ['path', 'name', 'title', 'description', 'export_module']
  Signature: class NotebookInfo:...

DirectoryInfo:
  Decorators: ['dataclass']
  Methods: ['total_notebook_count']
  Attributes: ['path', 'name', 'notebook_count', 'description', 'subdirs', 'notebooks']
  Signature: class DirectoryInfo:...


In [ ]:
# Test extracting function signatures
print("Function signatures with docments:")
for func in core_info.functions[:2]:
    print(f"\n{func.name}:")
    print(func.signature)

Function signatures with docments:

get_notebook_files:
def get_notebook_files(path: Path = None,           # Directory to search (defaults to nbs_path)
                      recursive: bool = True        # Search subdirectories
                      ) -> List[Path]:              # List of notebook paths

get_subdirectories:
def get_subdirectories(path: Path = None,           # Directory to search (defaults to nbs_path)
                      recursive: bool = False       # Include all nested subdirectories
                      ) -> List[Path]:              # List of directory paths


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()